# Aplicaciones de PCA en Electrónica

## Análisis de Componentes Principales para Problemas Electrónicos

Este notebook presenta aplicaciones prácticas del Análisis de Componentes Principales (PCA) en el campo de la electrónica, incluyendo:

1. **Eliminación de ruido en señales electrónicas**
2. **Detección de fallos en circuitos**
3. **Análisis de imágenes de componentes electrónicos**

---


#### Importación de Librerías y Configuración


In [ ]:
# Cargar las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

# Configuración de matplotlib
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.style.use('seaborn-v0_8')

print("Librerías importadas correctamente")

## Aplicación 1: Limpieza de ruido de señales electrónicas

### Problema:
Señales electrónica contaminada con ruido degradando la calidad de la información.

### Solución con PCA:
Usar PCA para separar la señal útil del ruido, manteniendo solo los componentes principales.


In [ ]:
# Generar señal sintética con ruido
def generar_senales_electronica(t, frecuencia=50, amplitud=1, ruido=0.3):
    # Señal principal (seno)
    senal_limpia = amplitud * np.sin(2 * np.pi * frecuencia * t)

    # Armónicos
    armonico_1 = 0.3 * amplitud * np.sin(2 * np.pi * frecuencia * t * 3)
    armonico_2 = 0.1 * amplitud * np.sin(2 * np.pi * frecuencia * t * 5)

    # Ruido gaussiano
    ruido_gaussiano = ruido * np.random.normal(0, 1, len(t))  # m = 0, sigma = 1

    # Ruido de alta frecuencia
    ruido_af = 0.2 * ruido * amplitud * np.sin(2 * np.pi * frecuencia * t * 50)

    senal_ruidosa = senal_limpia + armonico_1 + armonico_2 + ruido_gaussiano + ruido_af

    return senal_limpia, senal_ruidosa

# Parámetros de la señal
tiempo = np.linspace(0, 1, 1000)            # 1000 muestras en 1 segundo
senal_limpia, senal_ruidosa = generar_senales_electronica(tiempo)

# Análisis espectral
fft_limpia = np.fft.fft(senal_limpia)                               # Transformada de Fourier de la señal limpia
fft_ruidosa = np.fft.fft(senal_ruidosa)                             # Transformada de Fourier de la señal ruidosa
frecuencias = np.fft.fftfreq(len(tiempo), tiempo[1] - tiempo[0])    # Frecuencias correspondientes a la FFT

In [ ]:
# Tamño de las señales
print(f"    Señal limpia: {senal_limpia.shape}, Señal ruidosa: {senal_ruidosa.shape}")

In [ ]:
# Visualización de las señales
def graficos():
    plt.figure(figsize=(15, 10))

    plt.subplot(2, 2, 1)
    plt.plot(tiempo, senal_limpia, 'b-', linewidth=2, label='Señal limpia')
    plt.plot(tiempo, senal_ruidosa, 'r-', alpha=0.7, label='Señal ruidosa')
    plt.title('Señal Electrónica: Original vs Con Ruido')
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Amplitud')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(2, 2, 2)
    plt.plot(frecuencias[:len(frecuencias)//2], np.abs(fft_limpia[:len(frecuencias)//2]),  'b-', label='Espectro Original')
    plt.plot(frecuencias[:len(frecuencias)//2], np.abs(fft_ruidosa[:len(frecuencias)//2]), 'r-', alpha=0.7, label='Espectro con Ruido')
    plt.title('Análisis Espectral')
    plt.xlabel('Frecuencia (Hz)')
    plt.ylabel('Magnitud')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"SNR (Signal-to-Noise Ratio) estimado: {20*np.log10(np.std(senal_limpia)/np.std(senal_ruidosa - senal_limpia)):.2f} dB")
    print(f"Relación entre la potencia de la señal y la potencia del ruido")
graficos()

In [ ]:
# Aplicar PCA para reducir el ruido en señales electrónicas
def reducir_ruido_con_pca(senal, n_ventanas=50, n_componentes=5):
    # Crear ventanas deslizantes para PCA
    ventana_size = len(senal) // n_ventanas # 50 ventanas de 20 muestras cada una
    ventanas = []                           # lista para almacenar las ventanas
    for i in range(n_ventanas):
        inicio = i * ventana_size
        fin = inicio + ventana_size
        if fin <= len(senal):
            ventanas.append(senal[inicio:fin])

    # Convertir a matriz para PCA
    matriz_ventanas = np.array(ventanas)    # cada fila es una ventana

    # Aplicar PCA
    pca = PCA(n_components=n_componentes)   # crea modelo PCA
    pca.fit(matriz_ventanas)                # ajusta el modelo a los datos

    # Transformar y reconstruir
    senal_transformada = pca.transform(matriz_ventanas)     # reduce dimensionalidad

    # Reconstruye la señal en ventanas
    senal_reconstruida = pca.inverse_transform(senal_transformada)

    # Reconstruir la señal completa
    senal_filtrada = np.zeros_like(senal)
    for i, ventana in enumerate(senal_reconstruida):
        inicio = i * ventana_size
        fin = inicio + ventana_size
        if fin <= len(senal):
            senal_filtrada[inicio:fin] = ventana

    print(f"Señal: {senal.shape} -> {matriz_ventanas.shape} -> {senal_transformada.shape} ->"
          f"{senal_reconstruida.shape} -> {senal_filtrada.shape}")

    return senal_filtrada, pca.explained_variance_ratio_

senal_filtrada, varianza_explicada = reducir_ruido_con_pca(senal_ruidosa)

In [ ]:
# Para hacer pruebas


In [ ]:
# Visualización de los resultados
def graficos():
    plt.figure(figsize=(15, 12))

    plt.subplot(3, 2, 1)
    plt.plot(tiempo, senal_limpia,  'b-', linewidth=2, label='Señal Original')
    plt.plot(tiempo, senal_ruidosa, 'r:', alpha=0.5,   label='Señal con Ruido')
    plt.title('Comparación de Señales')
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Amplitud')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(3, 2, 2)
    plt.plot(tiempo, senal_limpia,   'b-', linewidth=2, label='Señal Original')
    plt.plot(tiempo, senal_filtrada, 'g:', linewidth=2, label='Señal Filtrada con PCA')
    plt.title('Resultado del Filtrado PCA')
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Amplitud')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(3, 2, 3)
    plt.plot(tiempo, senal_ruidosa - senal_limpia,  'r-', alpha=0.7, label='Ruido Original')
    plt.plot(tiempo, senal_filtrada - senal_limpia, 'g:', alpha=0.7, label='Ruido Residual')
    plt.title('Análisis del Ruido')
    plt.xlabel('Tiempo (s)')
    plt.ylabel('Amplitud del Ruido')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(3, 2, 4)
    plt.bar(range(1, len(varianza_explicada)+1), varianza_explicada, color='blue')
    plt.title('Varianza Explicada por Componentes')
    plt.xlabel('Componente Principal')
    plt.ylabel('Varianza Explicada')
    plt.grid(True, alpha=0.3)

    # Análisis espectral comparativo
    fft_filtrada = np.fft.fft(senal_filtrada)

    plt.subplot(3, 2, 5)
    plt.plot(frecuencias[:len(frecuencias)//2], np.abs(fft_limpia[:len(frecuencias)//2]), 'b-', label='Original')
    plt.plot(frecuencias[:len(frecuencias)//2], np.abs(fft_ruidosa[:len(frecuencias)//2]), 'r-', alpha=0.5, label='Con Ruido')
    plt.plot(frecuencias[:len(frecuencias)//2], np.abs(fft_filtrada[:len(frecuencias)//2]), 'g:', label='Filtrada')
    plt.title('Espectros Comparativos')
    plt.xlabel('Frecuencia (Hz)')
    plt.ylabel('Magnitud')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Métricas de calidad
    snr_original = 20*np.log10(np.std(senal_limpia)/np.std(senal_ruidosa - senal_limpia))
    snr_filtrada = 20*np.log10(np.std(senal_limpia)/np.std(senal_filtrada - senal_limpia))
    mejora_snr = snr_filtrada - snr_original

    plt.subplot(3, 2, 6)
    metricas = ['SNR Original', 'SNR Filtrada', 'Mejora SNR']
    valores = [snr_original, snr_filtrada, mejora_snr]
    colores = ['blue', 'green', 'purple']
    plt.bar(metricas, valores, color=colores, alpha=0.7)
    plt.title('Métricas de Calidad')
    plt.ylabel('SNR (dB)')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"SNR Original: {snr_original:.2f} dB") # Relación señal/ruido original
    print(f"SNR Filtrada: {snr_filtrada:.2f} dB") # Relación señal/ruido después de aplicar PCA
    print(f"Mejora en SNR: {mejora_snr:.2f} dB")  # Mejora en la relación señal/ruido
    print(f"Varianza total explicada: {np.sum(varianza_explicada):.3f}") # Variabilidad total explicada por los componentes principales
graficos()

## Aplicación 2: Detección de Fallas en Circuitos Electrónicos

### Problema:
Identificar componentes defectuosos en circuitos electrónicos mediante análisis de señales de prueba.

### Solución con PCA:
Usar PCA para detectar anomalías en las características eléctricas de los componentes.


In [ ]:
# Simulación de datos de circuitos electrónicos
def generar_datos_circuito(n_muestras, falla_prob):
    # np.random.seed(42)

    # Características normales de un circuito
    datos_normales = []
    datos_fallas = []

    for _ in range(n_muestras):
        # Características base (resistencia, capacitancia, inductancia, etc.)
        resistencia = np.random.normal(100, 10)                 # 100 ohm ± 10%
        capacitancia = np.random.normal(1e-6, 1e-7)             # 1µF ± 10%
        inductancia = np.random.normal(1e-3, 1e-4)              # 1mH ± 10%
        frecuencia_resonancia = np.random.normal(1000, 100)     # 1kHz ± 10%
        impedancia = np.random.normal(50, 5)                    # 50 ohm ± 10%
        factor_calidad = np.random.normal(100, 10)              # Q factor ± 10%
        temperatura = np.random.normal(25, 5)                   # 25°C ± 5°C
        humedad = np.random.normal(50, 10)                      # 50% ± 5%
        voltaje_ruido = np.random.normal(0, 0.1)                # Ruido de voltaje ± 0.1V
        corriente_fuga = np.random.normal(1e-9, 1e-10)          # Corriente de fuga ± 1e-10A

        caracteristicas = [
            resistencia, capacitancia, inductancia, frecuencia_resonancia,
            impedancia, factor_calidad, temperatura, humedad,
            voltaje_ruido, corriente_fuga
        ]

        # Determinar si es una falla
        if np.random.random() < falla_prob:
            # Introducir fallas específicas
            falla_tipo = np.random.choice(['resistencia_alta', 'capacitancia_baja', 'ruido_excesivo'])

            if falla_tipo == 'resistencia_alta':
                caracteristicas[0] *= np.random.uniform(2, 5)       # Resistencia muy alta
            elif falla_tipo == 'capacitancia_baja':
                caracteristicas[1] *= np.random.uniform(0.1, 0.5)   # Capacitancia muy baja
            elif falla_tipo == 'ruido_excesivo':
                caracteristicas[8] *= np.random.uniform(5, 20)      # Ruido excesivo

            datos_fallas.append(caracteristicas + [1])              # 1 = falla
        else:
            datos_normales.append(caracteristicas + [0])            # 0 = normal

    return np.array(datos_normales), np.array(datos_fallas)

datos_normales, datos_fallas = generar_datos_circuito(200, 0.1)

# Combinar todos los datos
todos_datos = np.vstack([datos_normales, datos_fallas])
X = todos_datos[:, :-1]  # Características
y = todos_datos[:, -1]   # Etiquetas (0=normal, 1=falla)

# Nombres de las características
nombres_caracteristicas = [
    'Resistencia (Ω)', 'Capacitancia (F)', 'Inductancia (H)', 'F Resonancia (Hz)', 'Impedancia (Ω)',
    'Factor Calidad', 'Temperatura (°C)', 'Humedad (%)', 'Voltaje Ruido (V)', 'Corriente Fuga (A)'
]

print(f"Datos generados: {len(datos_normales)} normales, {len(datos_fallas)} con fallas")
print(f"Total de características: {X.shape[1]}")
print(f"Tasa de fallas: {len(datos_fallas)/len(todos_datos)*100:.1f}%")

In [ ]:
# Para hacer pruebas
datos_normales.shape, datos_fallas.shape, todos_datos.shape, X.shape, y.shape, len(todos_datos), len(X)

In [ ]:
# (x) Análisis exploratorio de datos (x)
def graficos():
    plt.figure(figsize=(15, 10))

    # Matriz de correlación
    plt.subplot(2, 3, 1)
    correlacion = np.corrcoef(X.T)
    im = plt.imshow(correlacion, cmap='coolwarm', aspect='auto')
    plt.colorbar(im)
    plt.title('Matriz de Correlación')
    plt.xlabel('Características')
    plt.ylabel('Características')

    # Distribución de características principales
    plt.subplot(2, 3, 2)
    plt.hist(X[y==0, 0], bins=20, alpha=0.7, label='Normal', color='blue')
    plt.hist(X[y==1, 0], bins=20, alpha=0.7, label='Falla', color='red')
    plt.xlabel('Resistencia (Ω)')
    plt.ylabel('Frecuencia')
    plt.title('Distribución de Resistencia')
    plt.legend()

    plt.subplot(2, 3, 3)
    plt.hist(X[y==0, 1], bins=20, alpha=0.7, label='Normal', color='blue')
    plt.hist(X[y==1, 1], bins=20, alpha=0.7, label='Falla', color='red')
    plt.xlabel('Capacitancia (F)')
    plt.ylabel('Frecuencia')
    plt.title('Distribución de Capacitancia')
    plt.legend()

    # Boxplot de características
    plt.subplot(2, 3, 4)
    datos_plot = [X[y==0, 8], X[y==1, 8]]  # Voltaje de ruido
    plt.boxplot(datos_plot, tick_labels=['Normal', 'Falla'])
    plt.ylabel('Voltaje Ruido (V)')
    plt.title('Distribución del Ruido')

    # Scatter plot de dos características
    plt.subplot(2, 3, 5)
    plt.scatter(X[y==0, 0], X[y==0, 1], alpha=0.6, label='Normal', color='blue')
    plt.scatter(X[y==1, 0], X[y==1, 1], alpha=0.6, label='Falla', color='red')
    plt.xlabel('Resistencia (Ω)')
    plt.ylabel('Capacitancia (F)')
    plt.title('Resistencia vs Capacitancia')
    plt.legend()

    # Estadísticas por tipo
    plt.subplot(2, 3, 6)
    estadisticas = ['Media', 'Desv. Est.', 'Min', 'Max']
    valores_normales = [np.mean(X[y==0, 0]), np.std(X[y==0, 0]), np.min(X[y==0, 0]), np.max(X[y==0, 0])]
    valores_fallas = [np.mean(X[y==1, 0]), np.std(X[y==1, 0]), np.min(X[y==1, 0]), np.max(X[y==1, 0])]

    x = np.arange(len(estadisticas))
    width = 0.35

    plt.bar(x - width/2, valores_normales, width, label='Normal', alpha=0.7, color='blue')
    plt.bar(x + width/2, valores_fallas, width, label='Falla', alpha=0.7, color='red')
    plt.xlabel('Estadísticas')
    plt.ylabel('Resistencia (Ω)')
    plt.title('Estadísticas de Resistencia')
    plt.xticks(x, estadisticas)
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Estadísticas descriptivas
    print("Estadísticas Descriptivas:")
    print("="*50)
    df_stats = pd.DataFrame(X, columns=nombres_caracteristicas)
    df_stats['Estado'] = ['Normal' if label == 0 else 'Falla' for label in y]
    print(df_stats.groupby('Estado').describe().round(3))
graficos()

In [ ]:
# Aplicar PCA para detección de fallas en circuitos electrónicos
def aplicar_pca_detección_fallas(X, y, n_componentes):
    # Normalizar los datos
    scaler = StandardScaler()
    X_normalizado = scaler.fit_transform(X)    # média = 0, desviación estándar = 1

    # Aplicar PCA
    pca = PCA(n_components=n_componentes)
    X_pca = pca.fit_transform(X_normalizado)

    # Crear DataFrame para análisis
    df_pca = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(n_componentes)])
    df_pca['Estado'] = ['Normal' if label == 0 else 'Falla' for label in y]

    print(f"X {X.shape} --> X_normalizado {X_normalizado.shape} --> X_pca {X_pca.shape}")

    return X_pca, pca, scaler, df_pca

X_pca, pca, scaler, df_pca = aplicar_pca_detección_fallas(X, y, 3)

In [ ]:
# para hacer pruebas
X_pca.shape, df_pca.shape, pca

In [ ]:
# (x) Visualización de los resultados del PCA (x)
def graficos():
    plt.figure(figsize=(15, 12))

    # Varianza explicada
    plt.subplot(2, 3, 1)
    plt.bar(range(1, len(pca.explained_variance_ratio_)+1), pca.explained_variance_ratio_, color=['blue', 'green','brown'])
    plt.xlabel('Componente Principal')
    plt.ylabel('Varianza Explicada')
    plt.title('Varianza Explicada por Componentes')
    plt.xticks(range(1, len(pca.explained_variance_ratio_)+1), [f'PC{i+1}' for i in range(len(pca.explained_variance_ratio_))], rotation=45)
    plt.grid(True, alpha=0.3)

    # Varianza acumulada
    plt.subplot(2, 3, 2)
    varianza_acumulada = np.cumsum(pca.explained_variance_ratio_)
    plt.plot(range(1, len(varianza_acumulada)+1), varianza_acumulada, 'bo-')
    plt.xticks(range(1, len(varianza_acumulada)+1), [f'PC{i+1}' for i in range(len(varianza_acumulada))], rotation=45)
    plt.xlabel('Número de Componentes')
    plt.ylabel('Varianza Acumulada')
    plt.title('Varianza Acumulada')
    plt.grid(True, alpha=0.3)

    # Scatter plot PC1 vs PC2
    plt.subplot(2, 3, 3)
    normal_mask = df_pca['Estado'] == 'Normal'
    falla_mask = df_pca['Estado'] == 'Falla'

    plt.scatter(df_pca[normal_mask]['PC1'], df_pca[normal_mask]['PC2'],
               alpha=0.6, label='Normal', color='blue')
    plt.scatter(df_pca[falla_mask]['PC1'], df_pca[falla_mask]['PC2'],
               alpha=0.6, label='Falla', color='red')
    plt.xlabel('Primera Componente Principal')
    plt.ylabel('Segunda Componente Principal')
    plt.title('PCA: PC1 vs PC2')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Scatter plot PC1 vs PC3
    plt.subplot(2, 3, 4)
    plt.scatter(df_pca[normal_mask]['PC1'], df_pca[normal_mask]['PC3'],
               alpha=0.6, label='Normal', color='blue')
    plt.scatter(df_pca[falla_mask]['PC1'], df_pca[falla_mask]['PC3'],
               alpha=0.6, label='Falla', color='red')
    plt.xlabel('Primera Componente Principal')
    plt.ylabel('Tercera Componente Principal')
    plt.title('PCA: PC1 vs PC3')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Scatter plot PC2 vs PC3
    plt.subplot(2, 3, 5)
    plt.scatter(df_pca[normal_mask]['PC2'], df_pca[normal_mask]['PC3'],
               alpha=0.6, label='Normal', color='blue')
    plt.scatter(df_pca[falla_mask]['PC2'], df_pca[falla_mask]['PC3'],
               alpha=0.6, label='Falla', color='red')
    plt.xlabel('Segunda Componente Principal')
    plt.ylabel('Tercera Componente Principal')
    plt.title('PCA: PC2 vs PC3')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # Contribución de características originales
    plt.subplot(2, 3, 6)
    componentes = pca.components_
    plt.imshow(componentes, cmap='coolwarm', aspect='auto')
    plt.colorbar()
    plt.xlabel('Características Originales')
    plt.ylabel('Componentes Principales')
    plt.title('Contribución de Características')
    plt.xticks(range(len(nombres_caracteristicas)), nombres_caracteristicas, rotation=90)

    plt.tight_layout()
    plt.show()

    print(f"Varianza explicada por los primeros {len(pca.explained_variance_ratio_)} componentes:")
    for i, var in enumerate(pca.explained_variance_ratio_):
        print(f"PC{i+1}: {var:.3f} ({var*100:.1f}%)")

    print(f"Varianza total explicada: {np.sum(pca.explained_variance_ratio_):.3f} ({np.sum(pca.explained_variance_ratio_)*100:.1f}%)")
graficos()

In [ ]:
# Detección de anomalías usando Isolation Forest en el espacio PCA
def detectar_anomalias_pca(X_pca, y, contamination=0.1):
    # Entrenar Isolation Forest
    iso_forest = IsolationForest(contamination=contamination, random_state=42)
    anomalias = iso_forest.fit_predict(X_pca)  # -1 = anomalía, 1 = normal

    # Convertir -1 (anomalía) a 1, 1 (normal) a 0
    predicciones = (anomalias == -1).astype(int)

    # Calcular métricas
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(y, predicciones) # Precisión del modelo

    return predicciones, accuracy, iso_forest

predicciones, accuracy, iso_forest = detectar_anomalias_pca(X_pca, y)

print(f"Precisión del modelo: {accuracy:.3f}")

## Aplicación 3: Compresión de Imágenes de Componentes Electrónicos

### Problema:
Inspección automática de componentes electrónicos mediante análisis de imágenes para detectar defectos de fabricación.

### Solución con PCA (parte de la solucíon):
Usar PCA para extraer características principales de las imágenes y clasificar componentes como buenos o defectuosos.


In [ ]:
# Definir cómo generar imágenes sintéticas de componentes electrónicos
def resistencia(tamaño=(64, 64)):
    # Crear imagen base
    imagen = np.zeros(tamaño)

    # tipo_componente == 'resistencia':
    centro_x, centro_y = tamaño[0]//2, tamaño[1]//2
    longitud = 40
    ancho = 8

    # Cuerpo de la resistencia
    y_inicio = centro_y - ancho//2
    y_fin = centro_y + ancho//2
    x_inicio = centro_x - longitud//2
    x_fin = centro_x + longitud//2

    imagen[y_inicio:y_fin, x_inicio:x_fin] = 1

    # Cables
    imagen[centro_y-2:centro_y+2, :x_inicio] = 1
    imagen[centro_y-2:centro_y+2, x_fin:] = 1

    # Añadir defectos si es defectuoso
    defectuoso = np.random.random() < 0.2  # 20% defectuosos
    if defectuoso:
        # Rayas o grietas
        if np.random.random() < 0.5:
            imagen[centro_y-1:centro_y+1, x_inicio+10:x_inicio+15] = 0
        else:
            # Mancha
            mancha_x = centro_x + np.random.randint(-10, 10)
            mancha_y = centro_y + np.random.randint(-5, 5)
            if 0 <= mancha_x < tamaño[0] and 0 <= mancha_y < tamaño[1]:
                imagen[mancha_y-2:mancha_y+2, mancha_x-2:mancha_x+2] = 0.5
    return imagen, defectuoso
def capacitor(tamaño=(64, 64)):
    # Crear imagen base
    imagen = np.zeros(tamaño)

    # Dibujar capacitor
    centro_x, centro_y = tamaño[0]//2, tamaño[1]//2
    ancho = 20
    alto = 30

    # Cuerpo del capacitor
    y_inicio = centro_y - alto//2
    y_fin = centro_y + alto//2
    x_inicio = centro_x - ancho//2
    x_fin = centro_x + ancho//2

    imagen[y_inicio:y_fin, x_inicio:x_fin] = 1

    # Cables
    imagen[centro_y-2:centro_y+2, :x_inicio] = 1
    imagen[centro_y-2:centro_y+2, x_fin:] = 1

    # Añadir defectos si es defectuoso
    defectuoso = np.random.random() < 0.2  # 20% defectuosos
    if defectuoso:
        # Deformación
        if np.random.random() < 0.5:
            imagen[y_inicio+5:y_fin-5, x_inicio+2:x_fin-2] = 0.8
        else:
            # Grietas
            imagen[y_inicio+10:y_inicio+12, x_inicio:x_fin] = 0
    return imagen, defectuoso
def inductor(tamaño=(64, 64)):
    # Crear imagen base
    imagen = np.zeros(tamaño)

    # Dibujar inductor (bobina)
    centro_x, centro_y = tamaño[0]//2, tamaño[1]//2
    radio = 15

    # Crear bobina circular
    y, x = np.ogrid[:tamaño[0], :tamaño[1]]
    distancia = np.sqrt((x - centro_x)**2 + (y - centro_y)**2)
    mascara = (distancia <= radio) & (distancia >= radio-3)
    imagen[mascara] = 1

    # Cables
    imagen[centro_y-2:centro_y+2, :centro_x-radio+1] = 1
    imagen[centro_y-2:centro_y+2, centro_x+radio:] = 1

    # Añadir defectos si es defectuoso
    defectuoso = np.random.random() < 0.2  # 20% defectuosos
    if defectuoso:
        # Interrupción en la bobina
        angulo = np.random.uniform(0, 2*np.pi)
        x_defecto = int(centro_x + (radio-1.5) * np.cos(angulo))
        y_defecto = int(centro_y + (radio-1.5) * np.sin(angulo))
        if 0 <= x_defecto < tamaño[0] and 0 <= y_defecto < tamaño[1]:
            imagen[y_defecto-1:y_defecto+1, x_defecto-1:x_defecto+1] = 0
    return imagen, defectuoso
def chip(tamaño=(64, 64)):
    # Crear imagen base
    imagen = np.zeros(tamaño)

    # Dibujar chip IC
    centro_x, centro_y = tamaño[0]//2, tamaño[1]//2
    ancho = 34
    alto = 20

    # Cuerpo del chip
    y_inicio = centro_y - alto//2
    y_fin = centro_y + alto//2
    x_inicio = centro_x - ancho//2 + 1
    x_fin = centro_x + ancho//2

    imagen[y_inicio:y_fin, x_inicio:x_fin] = 1

    # Pines
    for pin in range(8):
        pin_x = x_inicio + 2 + (pin * ancho) // 8
        imagen[y_inicio-3:y_inicio, pin_x-1:pin_x+1] = 1
        imagen[y_fin:y_fin+3, pin_x-1:pin_x+1] = 1

    # Añadir defectos si es defectuoso
    defectuoso = np.random.random() < 0.2  # 20% defectuosos
    if defectuoso:
        # Pin faltante
        pin_faltante = np.random.randint(0, 8)
        pin_x = x_inicio + 2 + (pin_faltante * ancho // 8)
        if pin_x < tamaño[0]:
            imagen[y_inicio-3:y_inicio, pin_x-1:pin_x+1] = 0
            imagen[y_fin:y_fin+3, pin_x-1:pin_x+1] = 0
    return imagen, defectuoso

In [ ]:
# Para hacer pruebas (4 componentes electrónicos)
def mostrar_imagenes_de_componentes():
    plt.figure(figsize=(15, 10))
    tipo_componente = ["Resistencia", "Capacitor", "Inductor", "Chip"]
    img_componente = [resistencia(), capacitor(), inductor(), chip()]
    for i in range(4):
        imagen, es_defectuoso = img_componente[i]
        plt.subplot(2, 4, i+1)
        plt.imshow(imagen, cmap='gray')
        plt.title(f'{tipo_componente[i]}')
        plt.xlabel(f'{"defectuoso" if es_defectuoso else "bueno"}')
        plt.xticks([]), plt.yticks([])
    plt.tight_layout()
    plt.show()
mostrar_imagenes_de_componentes()

In [ ]:
# Generar un conjunto de imágenes de componentes electrónicos
def generar_imagenes_componentes_electronicos(n_imagenes=100, tamaño=(64, 64)):
    np.random.seed(42)
    imagenes = []       # lista de vectores: cada vector es una imagen (matriz)
    etiquetas = []      # lista de enteros: 1 = defectuoso, 0 = bueno

    for i in range(n_imagenes):
        # Determinar tipo de componente
        tipo_componente = np.random.choice(['resistencia', 'capacitor', 'inductor', 'chip'])

        if tipo_componente == 'resistencia':
            imagen, es_defectuoso = resistencia()       # imagen (matriz), es_defectuoso (booleano)
        elif tipo_componente == 'capacitor':
            imagen, es_defectuoso = capacitor()
        elif tipo_componente == 'inductor':
            imagen, es_defectuoso = inductor()
        elif tipo_componente == 'chip':
            imagen, es_defectuoso = chip()

        # Añadir ruido
        ruido = np.random.normal(0, 0.1, tamaño)        # ruido (matriz), tamaño (tupla)
        imagen = np.clip(imagen + ruido, 0, 1)          # imagen (matriz) + ruido (matriz) --> imagen (matriz)

        imagenes.append(imagen.flatten())               # imagem (matriz) --> achatamiento --> vector
        etiquetas.append(1 if es_defectuoso else 0)     # 1 = defectuoso, 0 = bueno

    print(f"Imágenes generadas: {len(imagenes)}")
    print(f"Dimensiones de cada imagen: {tamaño[0]}x{tamaño[1]}")
    print(f"Formato: {len(imagenes)} x {tamaño} --> {len(imagenes)} x {tamaño[0]*tamaño[1]}")
    print(f"Componentes defectuosos: {np.sum(etiquetas)} ({np.sum(etiquetas)/len(etiquetas)*100:.1f}%)")
    print(f"Componentes buenos: {len(etiquetas) - np.sum(etiquetas)} ({(len(etiquetas) - np.sum(etiquetas))/len(etiquetas)*100:.1f}%)")

    return np.array(imagenes), np.array(etiquetas)      # imagenes (100x4096), etiquetas (100)
imagenes_componentes, etiquetas_componentes = generar_imagenes_componentes_electronicos()

In [ ]:
# Para hacer pruebas


In [ ]:
# Visualizar 12 imágenes aleatorias
def graficos():
    plt.figure(figsize=(15, 10))

    # # Mostrar ejemplos de imágenes
    # for i in range(12):
    #     plt.subplot(3, 4, i+1)
    #     imagen_ejemplo = imagenes_componentes[i].reshape(64, 64)
    #     plt.imshow(imagen_ejemplo, cmap='gray')
    #     plt.title(f'Imagen {i+1} - {"Defectuoso" if etiquetas_componentes[i] else "Bueno"}')
    #     plt.axis('off')

    # Mostrar 12 imágenes aleatorias de las 100 disponibles
    indices_aleatorios = np.random.choice(range(len(imagenes_componentes)), 12, replace=False)
    for i, idx in enumerate(indices_aleatorios):
        plt.subplot(3, 4, i+1)
        imagen_ejemplo = imagenes_componentes[idx].reshape(64, 64)
        plt.imshow(imagen_ejemplo, cmap='gray')
        plt.title(f'Imagen {idx+1} - {"Defectuoso" if etiquetas_componentes[idx] else "Bueno"}')
        plt.axis('off')

    plt.tight_layout()
    plt.show()
graficos()

In [ ]:
# Aplicar PCA a las imágenes de componentes electrónicos (50 -> 100 CP)
def aplicar_pca_imagenes(imagenes, n_componentes=50):
    # Normalizar imágenes
    scaler = StandardScaler()   # média = 0, desviación estándar = 1
    imagenes_normalizadas = scaler.fit_transform(imagenes)  # normaliza cada columna

    # Aplicar PCA
    pca = PCA(n_components=n_componentes)                   # crea modelo PCA
    imagenes_pca = pca.fit_transform(imagenes_normalizadas) # reduce dimensionalidad

    print(f"Datos: {len(imagenes)} x {imagenes_componentes.shape[1]} -→ {len(imagenes)} x {imagenes_pca.shape[1]}")
    print(f"Reducción de dimensionalidad: {imagenes_componentes.shape[1]} -→ {imagenes_pca.shape[1]}")
    print(f"Varianza explicada: {np.sum(pca.explained_variance_ratio_):.3f}")
    print(f"Compresión: {imagenes_componentes.shape[1]/imagenes_pca.shape[1]:.1f}x")

    return imagenes_pca, pca, scaler
imagenes_pca, pca_imagenes, scaler = aplicar_pca_imagenes(imagenes_componentes) # (50->100)

In [ ]:
# Visualizar resultados del PCA en imágenes
def graficos():
    plt.figure(figsize=(15, 12))

    # Varianza explicada
    plt.subplot(2, 3, 1)
    plt.bar(range(1, len(pca_imagenes.explained_variance_ratio_)+1),
            pca_imagenes.explained_variance_ratio_)
    plt.xlabel('Componente Principal')
    plt.ylabel('Varianza Explicada')
    plt.title('Varianza Explicada por Componentes')
    plt.grid(True, alpha=0.3)

    # Reconstruir algunas imágenes usando PCA
    plt.subplot(2, 3, 2)
    imagen_original = imagenes_componentes[0].reshape(64, 64)
    plt.imshow(imagen_original, cmap='gray')
    plt.title('Imagen Original')
    plt.axis('off')

    # Reconstruir imagen
    imagen_reconstruida = pca_imagenes.inverse_transform(imagenes_pca[0:1])
    imagen_reconstruida = scaler.inverse_transform(imagen_reconstruida)
    imagen_reconstruida = imagen_reconstruida.reshape(64, 64)

    plt.subplot(2, 3, 3)
    plt.imshow(imagen_reconstruida, cmap='gray')
    plt.title('Imagen Reconstruida')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    # Calcular métricas de reconstrucción
    error_reconstruccion = np.mean((imagenes_componentes -
                                   scaler.inverse_transform(
                                       pca_imagenes.inverse_transform(imagenes_pca)))**2)

    print(f"Métricas de PCA en imágenes:")
    print(f"Error de reconstrucción MSE: {error_reconstruccion:.6f}")
    print(f"Varianza total explicada: {np.sum(pca_imagenes.explained_variance_ratio_):.3f}")
    print(f"Compresión lograda: {imagenes_componentes.shape[1]/imagenes_pca.shape[1]:.1f}x")
    print(f"Reducción de dimensionalidad: {imagenes_componentes.shape[1]} → {imagenes_pca.shape[1]}")
graficos()

In [ ]:
# Para hacer pruebas


## Recomendaciones Prácticas

### Resumen de Aplicaciones

Hemos explorado tres aplicaciones de PCA en ingeniería electrónica:

1. **Eliminación de ruido en señales**: PCA puede reducir efectivamente el ruido en señales electrónicas manteniendo la información importante.

2. **Detección de Fallas**: PCA combinado con algoritmos de detección de anomalías puede identificar componentes defectuosos en circuitos.

3. **Análisis de Imágenes**: PCA puede extraer características principales de imágenes de componentes para inspección automática.

### Ventajas del PCA en Electrónica

- **Eficiencia computacional**: Reduce la complejidad de los algoritmos
- **Robustez**: Mejora la tolerancia al ruido
- **Interpretabilidad**: Facilita la comprensión de los datos
- **Escalabilidad**: Funciona con grandes volúmenes de datos
- **Versatilidad**: Se aplica a diferentes tipos de datos electrónicos

### Limitaciones y Consideraciones

- **Pérdida de información**: Al reducir dimensiones se puede perder información
- **Linealidad**: PCA asume relaciones lineales entre variables
- **Sensibilidad a outliers**: Los valores extremos pueden afectar los resultados
- **Interpretación**: Los componentes principales pueden ser difíciles de interpretar

### Recomendaciones Prácticas

- **Preprocesamiento**: Siempre normalizar los datos antes de aplicar PCA
- **Selección de componentes**: Usar el criterio de varianza explicada (95% es común)
- **Validación**: Verificar que la reducción de dimensionalidad no afecte la calidad
- **Visualización**: Usar las primeras componentes para visualización 2D/3D
- **Combinación**: PCA funciona bien combinado con otros algoritmos de ML

### Casos de Uso Reales

**1. Sistemas de Monitoreo Industrial**
- Monitoreo de vibraciones en motores eléctricos
- Análisis de temperatura en transformadores
- Detección de fallas en sistemas de potencia

**2. Dispositivos IoT y Sensores**
- Compresión de datos en sensores remotos
- Análisis de patrones en redes de sensores
- Optimización de transmisión de datos

**3. Control de Calidad en Manufactura**
- Inspección automática de componentes
- Detección de defectos en PCB
- Análisis de calidad en líneas de producción

**4. Sistemas de Diagnóstico**
- Diagnóstico de fallas en equipos médicos
- Análisis de señales biomédicas
- Monitoreo de sistemas críticos

### Métricas de Evaluación

- **Varianza explicada**: Porcentaje de información conservada
- **Error de reconstrucción**: Diferencia entre datos originales y reconstruidos
- **Ratio de compresión**: Reducción en el número de dimensiones
- **Tiempo de procesamiento**: Eficiencia computacional

### Extensiones y Variantes

- **Kernel PCA**: Para datos no lineales
- **Incremental PCA**: Para datos en streaming
- **Sparse PCA**: Para datos con muchas características irrelevantes
- **Robust PCA**: Para datos con outliers

## Referencias y Recursos Adicionales

- Scikit-learn Documentation:

  https://scikit-learn.org/stable/modules/decomposition.html#pca
- Pattern Recognition and Machine Learning - Christopher Bishop
- The Elements of Statistical Learning - Hastie, Tibshirani, Friedman
- Papers sobre PCA en aplicaciones electrónicas y de ingeniería
